# ¡Ejercicio de Gradio!

Agregaremos modelos disponibles de Llama como llama3.2, qwen2.5:7b, gemma3:27b,
qwq:latest ademas del gpt-minio


## ¡Prepárate para la alegría!

Ten en cuenta que las pantallas de Gradio pueden aparecer en "modo oscuro" o "modo claro" según la configuración de tu computadora.

In [60]:
# imports

import os
import requests
#from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
#import json
import sys           # Ya viene con Python
#import time          # Ya viene con Pytho
try:
    from IPython.display import Markdown, display,clear_output
    from io import StringIO
    IN_JUPYTER = True
except ImportError:
    IN_JUPYTER = False

In [61]:
import gradio as gr # oh yeah!

In [62]:
# Cargar variables de entorno en un archivo llamado .env
# Imprimir los prefijos de clave para facilitar la depuración

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")
else:
    print("OpenAI API Key Sin Configurar")
    
if anthropic_api_key:
    print(f"Anthropic API Key existe y empieza por {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key Sin Configurar")

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

OpenAI API Key existe y empieza por sk-proj-
Anthropic API Key existe y empieza por sk-ant-
Google API Key existe y empieza por AIzaSyBz


In [63]:
OLLAMA_API = "http://132.248.32.20:8080/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "ollama3.2"
messages = []

In [64]:
# Conéctese a OpenAI, Anthropic y Google; comente las líneas de Claude o Google si no las está usando

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [65]:
# Un mensaje genérico del sistema: ¡no más IA adversarias sarcásticas!

system_message = "Eres un asistente útil"

## Momento de crear nuestra interfaz de usuario

## Forzar el modo oscuro

Gradio aparece en modo claro o en modo oscuro según la configuración del navegador y la computadora. Hay una manera de forzar que Gradio aparezca en modo oscuro, pero Gradio recomienda no hacerlo ya que debería ser una preferencia del usuario (particularmente por razones de accesibilidad). Pero si desea forzar el modo oscuro para sus pantallas, a continuación se muestra cómo hacerlo.

In [66]:
# Define esta variable y luego pasa js=force_dark_mode al crear la interfaz

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""


In [67]:
def add_message(role, content, reset=False):
    """
    Agrega un mensaje al historial. Puede reiniciar la conversación si se indica.

    Args:
        role (str): 'system', 'user' o 'assistant'
        content (str): texto del mensaje
        reset (bool): si True, reinicia el historial antes de agregar
    """
    global messages
    if reset:
        messages = []
    if role not in ['system', 'user', 'assistant']:
        raise ValueError("El rol debe ser 'system', 'user' o 'assistant'")
    messages.append({"role": role, "content": content})

In [68]:
# Vamos a crear una llamada que transmita resultados
# Si deseas un repaso de los generadores (la palabra clave "yield"),
# Echa un vistazo al cuaderno de Python intermedio en la carpeta de la semana 1.


def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [69]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [70]:
def stream_ollama():
    """
    Envía el historial al modelo con stream=True y muestra salida tipo máquina de escribir.
    También devuelve y muestra el resultado completo en formato JSON.
    """
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }

    with requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True) as response:
        response.raise_for_status()
        full_text = ""

        for line in response.iter_lines(decode_unicode=True):
            if not line:
                continue
            # Cada línea en streaming es un JSON (por chunk)
            chunk = json.loads(line)
            delta = chunk.get("message", {}).get("content", "")
            full_text += delta
            yield full_text
            for char in delta:
                sys.stdout.write(char)
                sys.stdout.flush()
                #time.sleep(0.01)  # velocidad de "máquina de escribir"

        

In [86]:

def stream_gemini(prompt):    
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_message,
    )
    # Llamada al modelo Gemini con streaming
    chat = gemini.start_chat()
    response = chat.send_message(prompt, stream=True)
    result = ""
    for chunk in response:
        result += chunk.text
        yield result


In [87]:
def stream_model(prompt, model):
    print ("modelo ",model)
    global MODEL
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Llama3.2":
        MODEL = "llama3.2"
        add_message("system", system_message, reset=True)
        add_message("user", prompt)
        result = stream_ollama()
    elif model=="qwen2.5:7b":
        MODEL = "qwen2.5:7b"
        add_message("system", system_message, reset=True)
        add_message("user", prompt)
        result = stream_ollama()
    elif model=="gemma3:27b":
        MODEL = "gemma3:27b"
        add_message("system", system_message, reset=True)
        add_message("user", prompt)
        result = stream_ollama()
    elif model=="qwq:latest":
        MODEL = "qwq:latest"
        add_message("system", system_message, reset=True)
        add_message("user", prompt)
        result = stream_ollama()
    elif model=="gemini":
        add_message("system", system_message, reset=True)
        add_message("user", prompt)
        result = stream_gemini(prompt)
    else:
        raise ValueError("Modelo Desconocido")
    yield from result

In [88]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Tu mensaje:"), gr.Dropdown(["GPT", "Llama3.2","qwen2.5:7b","gemma3:27b","qwq:latest","gemini"], label="Selecciona un modelo:", value="GPT")],
    outputs=[gr.Markdown(label="Respuesta:")],
    js=force_dark_mode,
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.


In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key=google_api_key)

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("The opposite of hot is")
print(response.text)